In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, RandomForestRegressor
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

col_names = ['age', 'workclass', 'fnlwgt','education', 'education-num', 
'marital-status', 'occupation', 'relationship', 'race', 'sex',
'capital-gain','capital-loss', 'hours-per-week','native-country', 'income']

df = pd.read_csv('adult.data.csv', header=None, names = col_names)







    
#Find the best accuracy and at what depth that occurs



#Plot the accuracy scores for the test and train set over the range of depth values  


#Save the best random forest model and save the feature importances in a dataframe
best_rf = RandomForestClassifier(max_depth=None)



#Create two new features, based on education and native country
# df['education_bin'] = None

# feature_cols = ['age',
#        'capital-gain', 'capital-loss', 'hours-per-week', 'sex', 'race','education_bin']
#Use these two new additional features and recreate X and test/train split


#Find the best max depth now with the additional two features





#Save the best model and print the two features with the new feature set




The dataset has been loaded for you in script.py and saved as a dataframe named df. Some of the input and output features of interest are:

-`age`: continuous  
-`workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.  
-`education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th,   -Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool  
-`race`: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black  
-`sex`: Female, Male  
-`capital-gain`: continuous  
-`capital-loss`: continuous  
-`hours-per-week`: continuous  
-`native country`: discrete  
-`income`: discrete, >50K, <=50K  

In [56]:
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [59]:
#Distribution of income
df['income'].value_counts(normalize=True)

income
 <=50K    0.75919
 >50K     0.24081
Name: proportion, dtype: float64

In [60]:
#Clean columns by stripping extra whitespace for columns of type "object"

# Create list of columns that are objects
object_columns_list = df.select_dtypes(include=['object']).columns.tolist()

# Iterate through columns list and strip whitespace
for column in object_columns_list:
    df[column] = df[column].str.strip()

In [73]:
#Create feature dataframe X with feature columns and dummy variables for categorical features

# List of features to one hot encode
ohe_features = ['workclass', 'education', 'marital-status',
                'occupation', 'relationship', 'race', 'sex', 'native-country']

# One Hot Encode Categorical Columns that are objects
ohe = pd.get_dummies(df[ohe_features], drop_first=True)

# Create list of columns to drop and add income to list
columns_to_drop = ohe_features
columns_to_drop.append('income')

# Remove categorical features for later joining with ohe
df_no_categorical = df.drop(columns = columns_to_drop)

# Create Matrix X by joining df_no_categorical and ohe
X = df_no_categorical.join(ohe)

#Create output variable y which is binary, 0 when income is less than 50k, 1 when it is greather than 50k
y = np.where((df['income'] == '<=50K'), 0, 1) 

#Split data into a train and test set
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20)

In [76]:
#Instantiate random forest classifier, fit and score with default parameters
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

# Score model
rf_score = rf.score(x_test, y_test)
print(f'Baseline accuracy score: {rf_score}')

Baseline accuracy score: 0.8590511285122063


In [ ]:
#Tune the hyperparameter max_depth over a range from 1-25, save scores for test and train set
np.random.seed(0)
accuracy_train=[]
accuracy_test = []